In [ ]:
!pip install anthropic
!pip install -U google-genai

In [ ]:
import json
import os
import google.generativeai as genai
from dotenv import load_dotenv
import json
from pathlib import Path
from openai import OpenAI
from anthropic import Anthropic
import base64, mimetypes
from IPython.display import Image, display
from google import genai
from google.genai import types
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OPENAI_API_KEY=""
ANTHROPIC_API_KEY=""

In [ ]:
total_images = 0
dataset_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram based questions"
for chapter_folder in os.listdir(dataset_path):
  chapter_path = os.path.join(dataset_path,chapter_folder)
  print(chapter_folder,len(os.listdir(chapter_path)))
  total_images += len(os.listdir(chapter_path))
print(f"Total Images: {total_images}")


Chapter_11_Angular_Momentum_6 6
Chapter_13_Gravitation_1 1
Chapter_14_Fluid_Mechanics_5 5
Chapter_16_Waves_3 3
Chapter_17_Sound_7 7
Chapter_9_Linear_Momentum_and_Collisions_11 11
Chapter_8_Potential_Energy_and_Conservation_of_energy_8 8
Chapter_7_Work_and_Kinetic_Energy_5 5
Chapter_6_Applications_of_Newton_Laws_18 18
Chapter_5_Newton_laws_of_motion_12 12
Chapter_4_Motion_in_Two_and_Three_Dimension_2 2
Chapter_3_Motion_along_a_straight_line_4 4
Chapter_2_Vectors_4 4
Chapter_12_Static_Equilibrium_and_Elasticity_13 13
Chapter_10_Fixed_Axis_Rotation_15 15
Total Images: 114


In [ ]:
def generate_anthropic_answer(image_path="",THINKING_BUDGET=5000,MAX_TOKENS=5001):
  anthropicClient = Anthropic(api_key=ANTHROPIC_API_KEY)
  diagram_path = image_path
  mime, _ = mimetypes.guess_type(diagram_path)
  mime = mime or "image/png"
  CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
  with open(diagram_path, "rb") as f:
    b64 = base64.b64encode(f.read()).decode("utf-8")
  prompt = (
    "You are an expert physics professor. "
    "Solve the problem and all sub-parts step-by-step. "
    "For each part: write the key equation then substitute numbers and then compute. "
    "State assumptions if needed. "
    "Round your calculations to 2 decimal places. "
    "Be precise and concise—no extra commentary. "
    )
  msg_img = anthropicClient.messages.create(
      model=CLAUDE_MODEL,
      max_tokens=MAX_TOKENS,
      thinking={"type": "enabled", "budget_tokens": THINKING_BUDGET},
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "image",
                      "source": {"type": "base64", "media_type": mime, "data": b64},
                  },
                  {"type": "text", "text": prompt},
              ],
          }
      ],
  )

  output = "".join(
      block.text for block in msg_img.content if getattr(block, "type", None) == "text"
  )
  return output

In [ ]:
def generate_gpt_answer(image_path="",REASONING_EFFORT="low",VERBOSITY="low"):
  openAIClient = OpenAI(api_key=OPENAI_API_KEY)
  mime, _ = mimetypes.guess_type(image_path)
  mime = mime or "image/png"
  with open(image_path, "rb") as f:
      b64 = base64.b64encode(f.read()).decode("utf-8")
  data_url = f"data:{mime};base64,{b64}"

  prompt = (
    "You are an expert physics professor. "
    "Solve the problem and all sub-parts step-by-step. "
    "For each part: write the key equation then substitute numbers and then compute. "
    "State assumptions if needed. "
    "Round your calculations to 2 decimal places. "
    "Be precise and concise—no extra commentary. "
    )
  response = openAIClient.responses.create(
      model="gpt-5",
      input=[{
          "role": "user",
          "content": [
              {"type": "input_text",
              "text": prompt},
              {"type": "input_image",
              "image_url": data_url}
              ]
          }],
          reasoning={"effort": REASONING_EFFORT},
          text={"verbosity": VERBOSITY},
  )
  return response.output_text

In [ ]:
# Both queries on a single image takes 28s to return output on low reasoning effort and low verbosity
# generate_anthropic_answer("/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram based questions/Chapter_10_Fixed_Axis_Rotation_17/Screenshot 2025-10-07 at 10.17.27 AM.png")
# generate_gpt_answer("/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram based questions/Chapter_10_Fixed_Axis_Rotation_17/Screenshot 2025-10-07 at 10.17.27 AM.png")

# 114 Questions
# Need generated answers from both models
# Need to compare generated answers with ground truth manually
# Some questions have graph-based answer (multimodal answer).
# It is worth to check manually how these models attempt those questions
# While checking the answers manually, I’ll take screenshots of the answers and store them along with their respective questions.


## Anthropic Code

In [ ]:
start_time = time.time()
# File paths
output_file = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/anthropic_answers_diagram_based_questions.json"
dataset_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram based questions"

# Load existing answers if the file exists
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        all_answers = json.load(f)
else:
    all_answers = {}

for chapter_folder in os.listdir(dataset_path):
    chapter_path = os.path.join(dataset_path, chapter_folder)
    chapter_id = os.path.splitext(chapter_folder)[0]  # or just use chapter_folder if it's already clean

    # Initialize chapter section if not present
    if chapter_id not in all_answers:
        all_answers[chapter_id] = {}

    for question_img in os.listdir(chapter_path):
        question_id = os.path.splitext(question_img)[0]

        # Skip if already processed
        if question_id in all_answers[chapter_id]:
            continue

        question_path = os.path.join(chapter_path, question_img)
        generated_answer = generate_anthropic_answer(question_path)

        # Save answer directly as string
        all_answers[chapter_id][question_id] = generated_answer

        # Write incrementally after each question
        with open(output_file, "w") as f:
            json.dump(all_answers, f, indent=2)

        print(f"Saved answer for {chapter_id}, Question {question_id}")


end_time = time.time()
elapsed_time = end_time - start_time

# 📝 Print in a nice format (e.g., min:sec)
mins, secs = divmod(elapsed_time, 60)
print(f"\nTotal time taken: {int(mins)} min {secs:.2f} sec")


Saved answer for Chapter_11_Angular_Momentum_6, Question 43
Saved answer for Chapter_11_Angular_Momentum_6, Question 57
Saved answer for Chapter_11_Angular_Momentum_6, Question 65
Saved answer for Chapter_11_Angular_Momentum_6, Question 77
Saved answer for Chapter_11_Angular_Momentum_6, Question 95
Saved answer for Chapter_11_Angular_Momentum_6, Question 97
Saved answer for Chapter_13_Gravitation_1, Question 9
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 11
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 25
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 55
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 73
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 89
Saved answer for Chapter_16_Waves_3, Question 27
Saved answer for Chapter_16_Waves_3, Question 69
Saved answer for Chapter_16_Waves_3, Question 139
Saved answer for Chapter_17_Sound_7, Question 23
Saved answer for Chapter_17_Sound_7, Question 41
Saved answer for Chapter_17_Sound_7, Question

## GPT Code

In [ ]:
start_time = time.time()
# File paths
output_file = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/GPT_answers_diagram_based_questions.json"
dataset_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram based questions"

# Load existing answers if the file exists
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        all_answers = json.load(f)
else:
    all_answers = {}

# For testing / breaking early
# count = 2

for chapter_folder in os.listdir(dataset_path):
    chapter_path = os.path.join(dataset_path, chapter_folder)
    chapter_id = os.path.splitext(chapter_folder)[0]  # or just use chapter_folder if it's already clean

    # Initialize chapter section if not present
    if chapter_id not in all_answers:
        all_answers[chapter_id] = {}

    for question_img in os.listdir(chapter_path):
        question_id = os.path.splitext(question_img)[0]

        # Skip if already processed
        if question_id in all_answers[chapter_id]:
            continue

        question_path = os.path.join(chapter_path, question_img)
        generated_answer = generate_gpt_answer(question_path)

        # Save answer directly as string
        all_answers[chapter_id][question_id] = generated_answer

        # Write incrementally after each question
        with open(output_file, "w") as f:
            json.dump(all_answers, f, indent=2)

        print(f"Saved answer for {chapter_id}, Question {question_id}")
    #     count -= 1
    #     if count == 0:
    #       break
    # if count == 0:
    #       break

end_time = time.time()
elapsed_time = end_time - start_time

# 📝 Print in a nice format (e.g., min:sec)
mins, secs = divmod(elapsed_time, 60)
print(f"\nTotal time taken: {int(mins)} min {secs:.2f} sec")


Saved answer for Chapter_11_Angular_Momentum_6, Question 43
Saved answer for Chapter_11_Angular_Momentum_6, Question 57
Saved answer for Chapter_11_Angular_Momentum_6, Question 65
Saved answer for Chapter_11_Angular_Momentum_6, Question 77
Saved answer for Chapter_11_Angular_Momentum_6, Question 95
Saved answer for Chapter_11_Angular_Momentum_6, Question 97
Saved answer for Chapter_13_Gravitation_1, Question 9
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 11
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 25
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 55
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 73
Saved answer for Chapter_14_Fluid_Mechanics_5, Question 89
Saved answer for Chapter_16_Waves_3, Question 27
Saved answer for Chapter_16_Waves_3, Question 69
Saved answer for Chapter_16_Waves_3, Question 139
Saved answer for Chapter_17_Sound_7, Question 23
Saved answer for Chapter_17_Sound_7, Question 41
Saved answer for Chapter_17_Sound_7, Question

## Print Output File

In [ ]:
output_file = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/anthropic_answers_diagram_based_questions.json"

# Load the JSON file
with open(output_file, "r") as f:
    all_answers = json.load(f)

# Pretty print the content
for chapter_id, questions in all_answers.items():
    print(f"\n {chapter_id}")
    print("-" * (len(chapter_id) + 3))
    for qid, answer in questions.items():
        print(f"\nQ{qid}\n-------------------\n {answer}")


 Chapter_11_Angular_Momentum_6
--------------------------------

Q43
-------------------
 # Solution

## Part (a): Angular Momentum Expression

**Key Equation:** L = r × p

**Setup:**
- Position vector: r = -d î + y ĵ
- Velocity: v = (dy/dt) ĵ = -gt ĵ (falling downward)
- Momentum: p = mv = -mgt ĵ

**Calculation:**
L = r × p = (-d î + y ĵ) × (-mgt ĵ)
L = (-d î) × (-mgt ĵ) + (y ĵ) × (-mgt ĵ)
L = mgdt (î × ĵ) + 0
L = mgdt k̂

**Answer:** **L_z = mgdt** (pointing out of page)

---

## Part (b): Torque Around z-axis

**Key Equation:** τ = r × F

**Setup:**
- Position vector: r = -d î + y ĵ  
- Gravitational force: F = -mg ĵ

**Calculation:**
τ = r × F = (-d î + y ĵ) × (-mg ĵ)
τ = (-d î) × (-mg ĵ) + (y ĵ) × (-mg ĵ)
τ = mgd (î × ĵ) + 0
τ = mgd k̂

**Answer:** **τ_z = mgd** (pointing out of page)

---

## Part (c): Verification of τ = dL/dt

**From part (a):** L_z = mgdt

**Time derivative:**
dL_z/dt = mgd

**From part (b):** τ_z = mgd

**Answer:** **Yes, τ = dL/dt is satisfied.** The torque

In [ ]:
output_file = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/GPT_answers_diagram_based_questions.json"

# Load the JSON file
with open(output_file, "r") as f:
    all_answers = json.load(f)

# Pretty print the content
print("GPT ANSWERS")
for chapter_id, questions in all_answers.items():
    print(f"\n {chapter_id}")
    print("-" * (len(chapter_id) + 3))
    for qid, answer in questions.items():
        print(f"\nQ{qid}\n-------------------\n {answer}")

GPT ANSWERS

 Chapter_11_Angular_Momentum_6
--------------------------------

Q43
-------------------
 Assumptions: Dropped from rest at (-d, 0, 0). Gravity is uniform F = -mg ĵ. Motion is only in y, so x = -d is constant.

(a) Angular momentum about z-axis
- Key equation: L = r × p
- Substitute: r = (-d î + y ĵ), p = m v_y ĵ
- Compute: L = (-d î + y ĵ) × (m v_y ĵ) = -m d v_y k̂
Using v_y = -gt (from rest): L = m d g t k̂

(b) Torque about z-axis
- Key equation: τ = r × F
- Substitute: F = -mg ĵ
- Compute: τ = (-d î + y ĵ) × (-mg ĵ) = mgd k̂

(c) Time rate of change
- Key equation: dL/dt
- Compute: dL/dt = d/dt(m d g t k̂) = mgd k̂ = τ
Answer: Yes, τ = dL/dt.

Q57
-------------------
 Given:
- ra = 40,000.00 km (from Earth’s center)
- Perigee altitude = 200.00 km → rp = RE + 200.00 = 6371.00 + 200.00 = 6571.00 km
- va = 3.00 km/s
- μ = 3.9860×10^5 km^3/s^2

Assumption: Two-body Keplerian motion; energy conserved.

Key equation (energy conservation/vis-viva form between two r

## GPT Results
Total Questions: 114 <br>
Wrongly Answered: <br>
Chapter 11


## Anthropic Results
Total Questions: 114 <br>
Wrongly Answered: